In [2]:
%autosave 0

Autosave disabled


In [3]:
import requests

In [4]:
url = "http://localhost:9696/predict"

In [8]:
house = {
        "mssubclass": 60,
        "mszoning": "RL",
        "lotfrontage": 65,
        "lotarea": 8450,
        "street": "pave",
        "alley": "na",
        "lotshape": "reg",
        "landcontour": "lvl",
        "utilities": "allpub",
        "lotconfig": "inside",
        "landslope": "gtl",
        "neighborhood": "collgcr",
        "condition1": "norm",
        "condition2": "norm",
        "bldgtype": "1fam",
        "housestyle": "2story",
        "overallqual": 7,
        "overallcond": 5,
        "yearbuilt": 2003,
        "yearremodadd": 2003,
        "roofstyle": "gable",
        "roofmatl": "compshg",
        "exterior1st": "vinylsd",
        "exterior2nd": "vinylsd",
        "masvnrtype": "brkface",
        "masvnrarea": 196,
        "exterqual": "gd",
        "extercond": "ta",
        "foundation": "pconc",
        "bsmtqual": "gd",
        "bsmtcond": "ta",
        "bsmtexposure": "no",
        "bsmtfintype1": "glq",
        "bsmtfinsf1": 706,
        "bsmtfintype2": "unf",
        "bsmtfinsf2": 0,
        "bsmtunfsf": 150,
        "totalbsmtsf": 856,
        "heating": "gasa",
        "heatingqc": "ex",
        "centralair": "y",
        "electrical": "sbrkr",
        "1stflrsf": 856,
        "2ndflrsf": 854,
        "lowqualfinsf": 0,
        "grlivarea": 1710,
        "bsmtfullbath": 1,
        "bsmthalfbath": 0,
        "fullbath": 2,
        "halfbath": 1,
        "bedroomabvgr": 3,
        "kitchenabvgr": 1,
        "kitchenqual": "gd",
        "totrmsabvgrd": 8,
        "functional": "typ",
        "fireplaces": 0,
        "fireplacequ": "na",
        "garagetype": "attchd",
        "garageyrblt": 2003,
        "garagefinish": "rfn",
        "garagecars": 2,
        "garagearea": 548,
        "garagequal": "ta",
        "garagecond": "ta",
        "paveddrive": "y",
        "wooddecksf": 0,
        "openporchsf": 61,
        "enclosedporch": 0,
        "3ssnporch": 0,
        "screenporch": 0,
        "poolarea": 0,
        "poolqc": "na",
        "fence": "na",
        "miscfeature": "na",
        "miscval": 0,
        "mosold": 2,
        "yrsold": 2008,
        "saletype": "wd",
        "salecondition": "normal"
}


In [14]:
requests.post(url, json=house).json()

{'price': 204622.625}